## How to inference Hugging Face models using Langchain

In [13]:
# install the necessay packages
! pip -q install langchain langchain_community

In [14]:
import os
from google.colab import userdata

# setting the Hugging Face Hub API token from user data
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HFKey')

In [15]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

# prompt template for evaluating answers
template = """Evaluate the following question and check whether the given answer is correct or not.

Question: {question}
Answer: {answer}

Provide the answer in a single word - YES or NO.
"""

# create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=template,
    input_variables=["question", "answer"]
)

# initialize the Hugging Face model endpoint for inference
model = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
# create a processing chain of the prompt and model
chain = prompt | model

question = "What is 2+2?"
answer = "5"

# invoke the chain with the question and answer, retrieving the result
result = chain.invoke({
    "question": f"{question}",
    "answer": f"{answer}"
})

In [17]:
print(result)



Solution: NO

The correct answer to the question "What is 2+2?" is 4, not 5. Therefore, the given answer is incorrect.





## Langchain with Pydantic Schema

In [18]:
from typing import List, Dict, Optional # for complex outputs
from pydantic import Field, BaseModel
from langchain.output_parsers import PydanticOutputParser

In [19]:
class EvaluationSchema(BaseModel):
    # This sets the default value as the text provided.
    # correct: str = Field("Check if the provided answer is correct or not - TRUE or FALSE.")

    correct: str = Field(description="Check if the provided answer is correct or not - TRUE or FALSE.")

In [20]:
class QuestionEvaluator:
    def __init__(self) -> None:

        # intialize the prompt template for evaluating questions and answers
        self.template = """
        Evaluate the following question and check whether the given answer is correct or not.

        Question: {question}
        Answer: {answer}

        {format_instructions}
        """

        # initialize the Hugging Face model endpoint for evaluation
        self.model = HuggingFaceEndpoint(
            repo_id="microsoft/Phi-3-mini-4k-instruct"
        )

        # initialize a parser for output
        self.parser = PydanticOutputParser(pydantic_object=EvaluationSchema)

    def get_chain(self):

        # retrieving format instructions from the parser
        format_instructions = self.parser.get_format_instructions()

        # creating a PromptTemplate instance with the evaluation template and input variables
        prompt = PromptTemplate(
            template=self.template,
            input_variables=[
                "question",
                "answer"
            ],
            partial_variables={
                "format_instructions": format_instructions
            }
        )

        # creating a processing chain that combines prompt, model, and parser
        chain = prompt | self.model | self.parser

        return chain

    def envoke_chain(self, question, answer):

        # load the processing chain
        chain = self.get_chain()

        # invoke the chain
        result = chain.invoke({
            "question": f"{question}",
            "answer": f"{answer}"
        })

        return result

In [21]:
question = "What is 2+2?"
answer = "5"

# create instance of QuestionEvaluator
evaluator = QuestionEvaluator()

# invoke the evaluation chain with the specified question and answer
result = evaluator.envoke_chain(
    question=question,
    answer=answer
).json()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
print(result)

{"correct":"FALSE"}
